In [4]:
pip install -r requirements.txt


  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.5 MB 6.7 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.5 MB 7.0 MB/s eta 0:00:02
   -------------- ------------------------- 4.2/11.5 MB 6.8 MB/s eta 0:00:02
   ------------------- -------------------- 5.5/11.5 MB 6.7 MB/s eta 0:00:01
   ------------------------ --------------- 7.1/11.5 MB 6.8 MB/s eta 0:00:01
   ----------------------------- ---------- 8.4/11.5 MB 6.8 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.5 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------  11.3/11

Downloading dependencies from 'Requirements.txt'

Loading the Libraries 

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings('ignore')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../input'